In [1]:
import timeit
import torch
import numpy as np

# import qpth.solvers.dynamic.solve as dynamic_solver
from qpth.util import get_sizes, extract_nBatch, expandParam, bdiag
from qpth.solvers import cvxpy, dynamic
from qpth.qp import QPFunction, QPSolvers

In [12]:
def get_kkt_problem(nBatch=2, nx=5, nineq=4, neq=3):
    def cast(m):
        # return m.cuda().double()
        return m.double()

    Q = cast(torch.randn(nx, nx))
    Q = Q.mm(Q.t())
    p = cast(torch.randn(nx))
    G = cast(torch.randn(nBatch, nineq, nx))
    h = cast(torch.randn(nBatch, nineq))
    A = cast(torch.randn(neq, nx))
    b = cast(torch.randn(neq))

    nBatch = extract_nBatch(Q, p, G, h, A, b)
    Q, _ = expandParam(Q, nBatch, 3)
    p, _ = expandParam(p, nBatch, 2)
    G, _ = expandParam(G, nBatch, 3)
    h, _ = expandParam(h, nBatch, 2)
    A, _ = expandParam(A, nBatch, 3)
    b, _ = expandParam(b, nBatch, 2)

    return Q, p, G, h, A, b

In [16]:
# Full Constraints

N_ITERS = 100

pdipm = QPFunction(verbose=-1, maxIter=20, solver=QPSolvers.PDIPM_BATCHED)
dynamic = QPFunction(verbose=-1, maxIter=20, solver=QPSolvers.DYNAMIC)
cvxpy = QPFunction(verbose=-1, maxIter=20, solver=QPSolvers.CVXPY)

num_same = 0
num_infeas = 0

time_p = 0
time_d = 0

showed = False

for _ in range(N_ITERS):
    Q, p, G, h, A, b = get_kkt_problem(1, 5, 4, 3)

    # Check for infeasibility with CVXPY
    try:
        zc = cvxpy(Q, p, G, h, A, b)
    except:
        num_infeas += 1
        continue

    start = timeit.default_timer()
    zp = pdipm(Q, p, G, h, A, b)

    mid = timeit.default_timer()
    zd = dynamic(Q, p, G, h, A, b)

    stop = timeit.default_timer()
    time_p += mid - start
    time_d += stop - start

    if np.allclose(zp, zd, rtol=1e-3):
        num_same += 1
        if not showed:
            showed = True
            print(Q, p, G, h)

print(num_same, num_infeas)
print(time_p, time_d)

0 29
0.7246376080220216 0.7977618609802448


In [17]:
# Ineq Constraints

N_ITERS = 100

pdipm = QPFunction(verbose=-1, maxIter=20, solver=QPSolvers.PDIPM_BATCHED)
dynamic = QPFunction(verbose=-1, maxIter=50, solver=QPSolvers.DYNAMIC_INEQ)
cvxpy = QPFunction(verbose=-1, maxIter=20, solver=QPSolvers.CVXPY)

num_same = 0
num_infeas = 0

time_p = 0
time_d = 0

showed = False

for _ in range(N_ITERS):
    Q, p, G, h, A, b = get_kkt_problem(1, 4, 3, 0)

    # Check for infeasibility with CVXPY
    try:
        zc = cvxpy(Q, p, G, h, A, b)
    except:
        num_infeas += 1
        continue

    start = timeit.default_timer()
    zp = pdipm(Q, p, G, h, A, b)

    mid = timeit.default_timer()
    zd = dynamic(Q, p, G, h, A, b)

    stop = timeit.default_timer()
    time_p += mid - start
    time_d += stop - start

    if np.allclose(zp, zd, rtol=1e-3):
        num_same += 1
        if not showed:
            showed = True
            print(Q, p, G, h)

print(num_same, num_infeas)
print(time_p, time_d)

tensor([[[ 2.3017, -1.8839,  0.5954,  0.3823],
         [-1.8839,  2.7673,  0.8585,  0.0732],
         [ 0.5954,  0.8585,  8.0163, -1.8710],
         [ 0.3823,  0.0732, -1.8710,  1.2067]]], dtype=torch.float64) tensor([[ 0.9158, -0.9019, -0.3382,  0.9116]], dtype=torch.float64) tensor([[[ 0.7071,  0.0434, -1.1788,  0.5716],
         [-2.3726,  2.2154,  0.1529,  0.1969],
         [ 0.0257, -2.2308, -1.3573, -0.4032]]], dtype=torch.float64) tensor([[ 0.9833,  0.2954, -0.0746]], dtype=torch.float64)
32 0
0.8351249179904698 0.9478432540272479
